In [1]:
import numpy as np
import rpy2
import rpy2.robjects as robjects
import pickle
from time import time

In [2]:
def store_results_dict(results, name):
    pickle.dump(results, open("./Results/{}.p".format(name), "wb" ))
    
def retrieve_results_dict(name):
    try:
        return pickle.load(open( "./Results/{}.p".format(name), "rb" ))
    except Exception as e:
        return None

### Data Generation

#### Raw Data

In [3]:
# GLOBAL CONFIG

# Var count
n_vars = 10

# Data types (default is standard normal)
binary_indeces = [1, 3, 6, 8, 9]
binarize = True

# Associations between vars an treat/outcome
treat_vars = [0,1,2,3,4,5,6,7]
outcome_vars = [0,1,2,3,4,8,9,10]

# Treat/outcome generation weights
assignment_weights = np.array([0, 0.8, -0.25, 0.6, -0.4, -0.8, -0.5, 0.7])
outcome_weights = np.array([-3.85, 0.3, -0.36, -0.73, -0.2, 0.71, -0.19, 0.26])
true_treat_effect = -0.4

def generate_data(n_samples=1000):
    # Generate 10 Random Vars
    # 1-4 are confounders: associated with outcome + treatment
    # 5-7 are exposure predictors
    # 8-10 are outcome predictors
    X = np.random.normal(loc=0.0, scale=1.0, size=(n_samples, n_vars))

    # Binarize specified vars if requested.
    if binarize:
        for var in binary_indeces:
            X[:, var-1] = (X[:, var -1] > 0).astype(int)

    # Add dummy for bias param     
    X = np.hstack([np.ones((n_samples, 1)), X])
    return X

In [4]:
# DEBUG
# X = generate_data(2000)
# X.shape

#### Assignment

In [5]:
# Create the models

assignment_models={}

def nonlinear_transform(X, B, quad_indeces):
    for quad_index in quad_indeces:
        quad = X[:, quad_index]**2
        X = np.hstack([X, quad.reshape(-1, 1)])
        B = np.append(B, B[quad_index])
    
    return X, B

def nonadditive_transform(X, B, interaction_indeces, interaction_weights=None):
    for interaction_index, var_indeces in enumerate(interaction_indeces):
        int_1, int_2 = var_indeces
        interaction_val = X[:, int_1]*X[:, int_2]
        
        if not interaction_weights:
            interaction_val = interaction_val*0.5
        else:
            interaction_val = interaction_val*interaction_weights[interaction_index]
            
        X = np.hstack([X, interaction_val.reshape(-1, 1)])
        B = np.append(B, B[int_1])
    
    return X, B

# Scenario 1
assignment_models["A_add_lin"] = lambda B, X: np.dot(X, B)

# Scenario 2:     
assignment_models["B_add_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2]))
# Scenario 3:
assignment_models["C_add_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(X, B,
                                                       quad_indeces=[2, 4, 7]))
# Scenario 4:
assignment_models["D_mild_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]))

# Scenario 5:
assignment_models["E_mild_nadd_mild_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (4,5), (5,6)]), quad_indeces=[2]))
# Scenario 6
assignment_models["F_mod_nadd_lin"] = lambda B, X: np.dot(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]))
# Scenario 7
assignment_models["G_mod_nadd_mod_nlin"] = lambda B, X: np.dot(*nonlinear_transform(*nonadditive_transform(X, B,
                                                       interaction_indeces=[(1,3), (2, 4), (3,5), (4,6), (5,7), (1,6), (2,3),
                                                                            (3,4), (4,5), (5,6)],
                                                       interaction_weights=[0.5, 0.7, 0.5, 0.7, 0.5, 0.5, 0.7, 0.5, 0.5, 0.5]), 
                                                                            quad_indeces=[2, 4, 7]))

In [6]:
# Tests 
assert(set(assignment_models["A_add_lin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([9, 37]))

assert(set(assignment_models["B_add_mild_nlin"](np.array([2, 0.5, 1.5]),
                                                np.array([[1, 2,4], [1, 10, 20]]))) == set([33, 637]))

assert(set(assignment_models["C_add_mod_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([373, 13457]))

assert(set(assignment_models["D_mild_nadd_lin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([139.5, 3632]))

assert(set(assignment_models["E_mild_nadd_mild_nlin"](np.array([2, 0.5, 1.5, 1, 1, 1, 2, 3]),
                                                np.array([[1, 2,4,5,6,7,8,9], [1, 10, 20, 30, 40, 50, 60, 60]]))) == set([163.5, 4232]))


#### Outcome and Assignment Functions

In [7]:
def get_assignments(B, X, n_samples, scenario="A_add_lin"):
    X_usable = X[:, treat_vars]
    
    # Calculate the probabilities of assignment
    linear_assignment_data = assignment_models[scenario](B, X_usable)
    p_treat = 1.0/(1+np.exp(-1*linear_assignment_data))

    # Assign
    rand = np.random.random(n_samples)
    assignments = (rand < p_treat).astype(int)
    
    return assignments

In [8]:
def get_outcomes(B, X, assignments, effect=true_treat_effect):
    X_usable = X[:, outcome_vars]
    return effect*assignments + np.dot(X_usable, B)

In [9]:
# DEBUG
# assignments = get_assignments(assignment_weights, X, "mild_nonaddititive_mild_nonlinear")
# outcomes = get_outcomes(outcome_weights, X, assignments)

### Analysis

In [10]:
from rpy2.robjects import IntVector, FloatVector, Formula
from rpy2.robjects.packages import importr
from rpy2.robjects import numpy2ri
numpy2ri.activate()

stats = importr('stats')
matching = importr('Matching')
snow = importr('snow')

Some code is going to >48 hours to ran. Lucky it's highly parellalisable so we can use a compute cluster. The one option is local to split across CPU cores. The better option is to go remote and explote 32 cores on multiple AWS machines.

On AWS, this is straightforward. Manually you need to port forward!. This allows the remote machine to connect to ports on the master via it's localhost loopback. 

```
# ~/.bash_profile
# Allow remote host to connect to local machine
# usage: $ remote_pfwd hostname {6000..6009}
function remote_pfwd {
  for i in ${@:2}
  do
    ssh -N -R $i:localhost:$i $1 &
  done
}
```
`remote_pfwd ubuntu@52.90.20.45 {11305..11307}`

In [11]:
AWS_MASTER_DNS="ip-172-31-42-147.ec2.internal"
AWS_SLAVE_1 = "ubuntu@ip-172-31-43-193.ec2.internal"
AWS_SLAVE_2 = "ubuntu@ip-172-31-81-244.ec2.internal"
AWS_MASTER_PORT_RANGE = list(range(11305, 11340))

class ClusterProvider(object):
    def __init__(self, n_nodes=8, remote_hosts=None, ports=None):
        if remote_hosts is None:
            self.cl = snow.makeSOCKcluster(["localhost"]*n_nodes)
        else:
            # Set the acceptable ports for connection
            # from the slaves
            if not ports:
                ports = AWS_MASTER_PORT_RANGE
            
            # Construct the connection string
            addresses = []
            for remote_host, n_nodes in remote_hosts:
                addresses+=[remote_host]*n_nodes
                
            self.cl = snow.makeSOCKcluster(addresses, rscript="Rscript", manual=False, snowlib="/usr/local/lib/R/site-library",
                                           port=IntVector(ports), master=AWS_MASTER_DNS, outfile="/dev/stdout", timeout=10)
    
    def get_cluster(self):
        return self.cl
    
    def kill_cluster(self):
        snow.stopCluster(self.cl)

In [12]:
# Local cluster
#cluster_provider = ClusterProvider(n_nodes=14)

In [12]:
# Remote cluster
cluster_provider = ClusterProvider(remote_hosts=[(AWS_SLAVE_1, 8)],
                                    ports = list(range(11305, 11314)))

In [25]:
# Run this with True to kill the cluster
kill = True
if kill:
    cluster_provider.kill_cluster()

#### Estimators

Define methods which can process outcomes, assignments and covariate data into a treatment effect estimate. 

1. Logistic Regression
2. GenMatch
3. VAE

In [13]:
def get_propensity_scores(assignments, covariate_data):
    # Setup
    y = IntVector(assignments)
    fmla = Formula('y ~ X')
    env = fmla.environment
    
    # Run propensiy regression
    env['X'] = covariate_data
    env['y'] = y
    fit = stats.glm(fmla, family="binomial")
    
    # DEBUG: fit.rx("coefficients")
    return fit.rx2("fitted.values")

In [14]:
# 1. Logisic Regression Propensity Matching
def logistic_prop_matching_est(outcomes, assignments, covariate_data):
    
    propensity_scores = get_propensity_scores(assignments, covariate_data)
    
    # Run matching
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=propensity_scores,
        replace=True)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [15]:
# 2. GenMatch Matching
def genmatch_est(outcomes, assignments, covariate_data):
    
    # Get the singleton cluster
    cl = cluster_provider.get_cluster()
    
    # Add prop scores to covar data
    propensity_scores = np.array(get_propensity_scores(assignments, covariate_data))
    matching_data = np.hstack([covariate_data, propensity_scores.reshape(-1, 1)])
    
    start = time()
    gen_out = matching.GenMatch(
        Tr=IntVector(assignments),
        X=matching_data,
        BalanceMatrix=covariate_data,
        print_level=0,
        cluster=cl)
    print("GenMatch Time: ", time() - start)
    
    match_out = matching.Match(
        Y=FloatVector(outcomes),
        Tr=IntVector(assignments),
        X=matching_data,
        replace=True,
        Weight_matrix=gen_out)
    
    return np.array(match_out.rx2("est").rx(1,1))[0]

In [16]:
# DEBUG
# est = logistic_prop_matching_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

In [17]:
# DEBUG
# est = genmatch_est(assignments, X[:, 1:]) # exclude the bias term
# np.array(est)

#### Monte Carlo Runner Code

In [18]:
def get_data(n_samples, assignment_model):
    X = generate_data(n_samples)
    assignments = get_assignments(assignment_weights, X,
                                  n_samples, assignment_model)

    outcomes = get_outcomes(outcome_weights, X, assignments)
    
    return assignments, outcomes, X

def get_estimate(outcomes, assignments, covar_data, method, **args):
    return method(outcomes, assignments, covar_data, **args)

In [19]:
def run_simulation(runs=1000, n_samples=1000,
                   assignment_model="additive_linear",
                   estimator=logistic_prop_matching_est,
                   verbose=True,
                   **args):
    
    progress_tick = max(1, int(runs/10))
    results = np.zeros(runs)

    for i in range(runs):
        assignments, outcomes, covar_data = get_data(n_samples, assignment_model)
        
        covar_data = covar_data[:, 1:] #exclude bias term
        results[i] = get_estimate(outcomes, assignments,
                                  covar_data, estimator, **args)
        
        if i%progress_tick == progress_tick-1 and verbose:
            print("Done {} of {}".format(i+1, runs))
    
    bias = np.abs(np.mean((true_treat_effect-results)/true_treat_effect)*100)
    rmse = np.mean((true_treat_effect-results)**2)**0.5
    
    if verbose:
        print("\nRMSE", rmse)
        print("Bias", bias)
        print("===============\n\n")
    
    return {"RMSE": rmse, "Bias": bias}

In [20]:
# DEBUG:
run_simulation(runs=20, n_samples=1000, assignment_model="D_mild_nadd_lin",
              estimator=genmatch_est, verbose=True)

/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: rgenoud

  warnings.warn(x, RRuntimeWarning)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: parallel

  warnings.warn(x, RRuntimeWarning)


GenMatch Time:  11.083404302597046
GenMatch Time:  31.12960457801819
Done 2 of 20
GenMatch Time:  11.981449365615845
GenMatch Time:  16.15269660949707
Done 4 of 20
GenMatch Time:  14.55371356010437
GenMatch Time:  13.442266702651978
Done 6 of 20
GenMatch Time:  14.062756061553955
GenMatch Time:  15.857194662094116
Done 8 of 20
GenMatch Time:  8.22305178642273
GenMatch Time:  12.44659686088562
Done 10 of 20
GenMatch Time:  10.008002996444702
GenMatch Time:  13.095482110977173
Done 12 of 20
GenMatch Time:  11.391975164413452
GenMatch Time:  15.019485712051392
Done 14 of 20
GenMatch Time:  10.63561201095581
GenMatch Time:  14.826309204101562
Done 16 of 20
GenMatch Time:  13.209826946258545
GenMatch Time:  12.118821620941162
Done 18 of 20
GenMatch Time:  17.138674020767212
GenMatch Time:  10.279953002929688
Done 20 of 20

RMSE 0.043918225298051665
Bias 0.3755681319660123




{'Bias': 0.3755681319660123, 'RMSE': 0.043918225298051665}

### Run MC trial

In [20]:
assignment_model_names = ['A_add_lin', 'B_add_mild_nlin', 'C_add_mod_nlin', 'D_mild_nadd_lin',
                     'E_mild_nadd_mild_nlin', 'F_mod_nadd_lin', 'G_mod_nadd_mod_nlin']

def get_store_name(models_being_run, est, runs, n_samples):
    # Storage
    
    if set(models_being_run) == set(assignment_model_names):
        store_name = "est_{}_runs_{}_n_{}".format(
            est.__name__,
            runs,
            n_samples)
    else:
        store_name = "est_{}_runs_{}_n_{}_models_{}".format(
            est.__name__,
            runs,
            n_samples,
            "_".join(models_being_run))
    
    return store_name

def run_test_battery(est,
                     store_name=None, 
                     runs=1000,
                     n_samples=1000,
                     models=assignment_models,
                     overwrite=False, verbosity=1,
                     **args):
    # Logging
    def printer(level, *args):
        if level <= verbosity:
            print(*args)
    
    # Storage config
    if store_name is None:
        store_name = get_store_name(models, est, runs, n_samples)
            
    results = retrieve_results_dict(store_name)

    if overwrite or (not results):
        printer(1, "No valid, existant results found. Beggining battery.\n")
        results = {}
        for model in models:
            printer(1, "Running: ", model)
            results[model] = run_simulation(
                                runs=runs,
                                n_samples=n_samples,
                                assignment_model=model,
                                estimator=est,
                                verbose=(verbosity==2),
                                **args)
            store_results_dict(results[model], store_name+"_checkpoint_"+model)
            printer(1, "Done.\n")

        store_results_dict(results, store_name)
    else:
        printer(1, "Displaying cached results.\n")
    
    printer(1, "Results")
    for model, results in results.items():
        printer(1, "Model: ", model)
        print(1, "Bias: ", results["Bias"])
        print(1, "RMSE: ", results["RMSE"], "\n")

### Run the Logistic Regression Battery

This one is easy. So we run on one machine.

In [21]:
run_test_battery(
    est=logistic_prop_matching_est,
    runs=1000,
    n_samples=1000)

Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  0.045874914703647685
1 RMSE:  0.07310500057973227 

Model:  B_add_mild_nlin
1 Bias:  3.1844355433209786
1 RMSE:  0.06588422028138122 

Model:  C_add_mod_nlin
1 Bias:  10.094350684204597
1 RMSE:  0.07650839711310455 

Model:  D_mild_nadd_lin
1 Bias:  6.720731771408928
1 RMSE:  0.08531717119502563 

Model:  E_mild_nadd_mild_nlin
1 Bias:  10.36168716658826
1 RMSE:  0.09094245826533698 

Model:  F_mod_nadd_lin
1 Bias:  3.1228082403965436
1 RMSE:  0.07605107262377982 

Model:  G_mod_nadd_mod_nlin
1 Bias:  11.830178367664905
1 RMSE:  0.07798212919046259 



### Run the GenMatch Battery

We split this across three machines using remote clusters.

In [23]:
gm_est = genmatch_est
gm_runs = 1000
gm_n_samples = 1000
gm_models_sets = [assignment_model_names[:3], assignment_model_names[3:5], assignment_model_names[5:]]
gm_files_to_be_produced = []

for model_set in gm_models_sets:
    gm_files_to_be_produced.append(get_store_name(model_set, gm_est, gm_runs, gm_n_samples))

gm_files_to_be_produced

['est_genmatch_est_runs_1000_n_1000_models_A_add_lin_B_add_mild_nlin_C_add_mod_nlin',
 'est_genmatch_est_runs_1000_n_1000_models_D_mild_nadd_lin_E_mild_nadd_mild_nlin',
 'est_genmatch_est_runs_1000_n_1000_models_F_mod_nadd_lin_G_mod_nadd_mod_nlin']

In [26]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[0],
    verbosity=2)

Displaying cached results.

Results
Model:  A_add_lin
1 Bias:  20.023235988165702
1 RMSE:  0.08009294395266281 

Model:  B_add_mild_nlin
1 Bias:  13.717727327601459
1 RMSE:  0.054870909310405835 

Model:  C_add_mod_nlin
1 Bias:  20.087548601352452
1 RMSE:  0.0803501944054098 



In [24]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[1],
    verbosity=2)

No valid, existant results found. Beggining battery.

Running:  D_mild_nadd_lin


/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: rgenoud

  warnings.warn(x, RRuntimeWarning)
/home/ubuntu/anaconda3/envs/fastai/lib/python3.6/site-packages/rpy2/rinterface/__init__.py:145: RRuntimeWarning: Loading required namespace: parallel

  warnings.warn(x, RRuntimeWarning)


GenMatch Time:  9.170192241668701
GenMatch Time:  16.475993394851685
GenMatch Time:  11.426947593688965
GenMatch Time:  8.873427391052246
GenMatch Time:  12.178423166275024
GenMatch Time:  11.628249406814575
GenMatch Time:  8.433069944381714
GenMatch Time:  15.911599159240723
GenMatch Time:  8.636510133743286
GenMatch Time:  17.13224482536316
GenMatch Time:  7.539711236953735
GenMatch Time:  22.462296724319458
GenMatch Time:  17.44847846031189
GenMatch Time:  8.520756006240845
GenMatch Time:  16.299620866775513
GenMatch Time:  6.77372932434082
GenMatch Time:  17.265355587005615
GenMatch Time:  8.790586233139038
GenMatch Time:  17.602644681930542
GenMatch Time:  10.677097797393799
GenMatch Time:  31.62210440635681
GenMatch Time:  17.161792755126953
GenMatch Time:  19.616565942764282
GenMatch Time:  15.366913557052612
GenMatch Time:  10.406134128570557
GenMatch Time:  18.57555866241455
GenMatch Time:  9.21586012840271
GenMatch Time:  14.24112319946289
GenMatch Time:  9.516291856765747
Ge

GenMatch Time:  28.134217262268066
GenMatch Time:  14.55392599105835
GenMatch Time:  13.614450454711914
GenMatch Time:  16.134591102600098
GenMatch Time:  7.452885866165161
GenMatch Time:  9.44089961051941
GenMatch Time:  13.547998666763306
GenMatch Time:  11.812290906906128
GenMatch Time:  18.38510489463806
GenMatch Time:  11.47507095336914
GenMatch Time:  17.56445598602295
GenMatch Time:  29.789704084396362
GenMatch Time:  14.593705654144287
GenMatch Time:  15.078595399856567
GenMatch Time:  8.234460353851318
GenMatch Time:  19.536216497421265
GenMatch Time:  20.15644931793213
GenMatch Time:  19.149520874023438
GenMatch Time:  22.010549068450928
GenMatch Time:  13.931457042694092
GenMatch Time:  14.779688119888306
GenMatch Time:  14.854763269424438
GenMatch Time:  7.197556734085083
GenMatch Time:  13.01790738105774
GenMatch Time:  8.817506551742554
GenMatch Time:  12.248228788375854
GenMatch Time:  4.645325183868408
GenMatch Time:  20.040923595428467
GenMatch Time:  14.7637619972229


GenMatch Time:  13.376500844955444
GenMatch Time:  32.74419355392456
GenMatch Time:  14.194806337356567
GenMatch Time:  17.783690452575684
GenMatch Time:  8.638848781585693
GenMatch Time:  7.5483012199401855
GenMatch Time:  10.784698724746704
GenMatch Time:  15.954810619354248
GenMatch Time:  25.064993143081665
GenMatch Time:  7.327942609786987
GenMatch Time:  10.84406065940857
GenMatch Time:  21.719139099121094
GenMatch Time:  8.26104187965393
GenMatch Time:  16.8612060546875
GenMatch Time:  29.102046251296997
GenMatch Time:  13.733819246292114
GenMatch Time:  22.620163202285767
GenMatch Time:  11.99048638343811
GenMatch Time:  7.387990474700928
GenMatch Time:  8.490526914596558
GenMatch Time:  13.054805040359497
GenMatch Time:  8.776494979858398
GenMatch Time:  31.48568367958069
GenMatch Time:  10.790832996368408
GenMatch Time:  14.022439241409302
GenMatch Time:  19.467543840408325
Done 500 of 1000
GenMatch Time:  17.426424741744995
GenMatch Time:  10.691348791122437
GenMatch Time:  

...

GenMatch Time:  16.262200593948364
GenMatch Time:  11.983081340789795
GenMatch Time:  12.513513326644897
GenMatch Time:  16.19461226463318
GenMatch Time:  21.654189348220825
GenMatch Time:  10.479152917861938
GenMatch Time:  7.7832560539245605
GenMatch Time:  17.484070301055908
GenMatch Time:  13.877406597137451
GenMatch Time:  8.46786904335022
GenMatch Time:  11.645203351974487
GenMatch Time:  12.664566278457642
GenMatch Time:  9.255668640136719
GenMatch Time:  15.628201007843018
GenMatch Time:  13.53504729270935
GenMatch Time:  12.25180983543396
GenMatch Time:  19.64311408996582
GenMatch Time:  33.96647095680237
GenMatch Time:  12.120949506759644
GenMatch Time:  12.59199070930481
GenMatch Time:  14.637908458709717
GenMatch Time:  10.97633957862854
GenMatch Time:  9.761385679244995
GenMatch Time:  16.646206378936768
GenMatch Time:  12.399325847625732
GenMatch Time:  21.580184936523438
GenMatch Time:  4.44724178314209
GenMatch Time:  14.982558727264404
GenMatch Time:  18.0385799407959


GenMatch Time:  17.00902533531189
GenMatch Time:  18.43159008026123
GenMatch Time:  14.434375524520874
GenMatch Time:  17.858801126480103
GenMatch Time:  14.507609128952026
GenMatch Time:  9.558912992477417
GenMatch Time:  14.484875202178955
GenMatch Time:  16.93858504295349
GenMatch Time:  21.071478128433228
GenMatch Time:  6.896181106567383
GenMatch Time:  14.532977819442749
GenMatch Time:  12.804308652877808
GenMatch Time:  7.171526908874512
GenMatch Time:  15.868540287017822
GenMatch Time:  8.165053129196167
GenMatch Time:  12.042840480804443
GenMatch Time:  9.258253335952759
GenMatch Time:  16.892223834991455
GenMatch Time:  12.780309677124023
GenMatch Time:  26.142335891723633
Done 200 of 1000
GenMatch Time:  11.922138214111328
GenMatch Time:  28.007977724075317
GenMatch Time:  13.764561891555786
GenMatch Time:  27.25610375404358
GenMatch Time:  15.530197143554688
GenMatch Time:  10.218993186950684
GenMatch Time:  7.899176120758057
GenMatch Time:  8.376412153244019
GenMatch Time:

GenMatch Time:  13.295143127441406
GenMatch Time:  11.595204591751099
GenMatch Time:  15.559452295303345
GenMatch Time:  7.6037211418151855
GenMatch Time:  22.150890350341797
GenMatch Time:  15.307656288146973
GenMatch Time:  9.462278366088867
GenMatch Time:  10.110558986663818
GenMatch Time:  10.321820974349976
GenMatch Time:  19.062318086624146
GenMatch Time:  15.269080400466919
GenMatch Time:  11.377534866333008
GenMatch Time:  13.513331651687622
GenMatch Time:  15.172677040100098
GenMatch Time:  9.53026032447815
GenMatch Time:  11.892541885375977
GenMatch Time:  12.716432094573975
GenMatch Time:  13.16477632522583
GenMatch Time:  20.585586071014404
GenMatch Time:  17.61706852912903
GenMatch Time:  25.40978193283081
GenMatch Time:  8.702529191970825
GenMatch Time:  8.872361660003662
GenMatch Time:  12.139321327209473
GenMatch Time:  19.46540904045105
GenMatch Time:  12.688720464706421
GenMatch Time:  17.576862335205078
GenMatch Time:  13.253417491912842
GenMatch Time:  16.1242465972

GenMatch Time:  10.036787271499634
GenMatch Time:  20.728089094161987
GenMatch Time:  11.58930230140686
GenMatch Time:  13.824543237686157
GenMatch Time:  15.608481168746948
GenMatch Time:  10.05946683883667
GenMatch Time:  8.294500350952148
GenMatch Time:  10.596662759780884
GenMatch Time:  11.854186773300171
GenMatch Time:  11.63593578338623
GenMatch Time:  16.7586088180542
GenMatch Time:  32.39038038253784
GenMatch Time:  12.646705150604248
GenMatch Time:  14.635969638824463
GenMatch Time:  7.643077611923218
GenMatch Time:  9.821375131607056
GenMatch Time:  15.262304306030273
GenMatch Time:  18.482434511184692
GenMatch Time:  21.597970247268677
GenMatch Time:  9.342440605163574
GenMatch Time:  10.71600341796875
GenMatch Time:  12.746000528335571
GenMatch Time:  8.566176891326904
GenMatch Time:  9.73409104347229
GenMatch Time:  14.051502466201782
GenMatch Time:  10.40467095375061
GenMatch Time:  14.830096960067749
GenMatch Time:  10.69855523109436
GenMatch Time:  13.131349563598633
G

GenMatch Time:  9.206010580062866
GenMatch Time:  7.697787284851074
GenMatch Time:  12.380228042602539
GenMatch Time:  17.22040605545044
GenMatch Time:  17.289698362350464
GenMatch Time:  14.634416580200195
GenMatch Time:  15.336341619491577
GenMatch Time:  14.029867887496948
GenMatch Time:  7.482014179229736
GenMatch Time:  15.621821403503418
Done 900 of 1000
GenMatch Time:  28.178414583206177
GenMatch Time:  27.183830738067627
GenMatch Time:  11.258795499801636
GenMatch Time:  15.508211374282837
GenMatch Time:  15.508097171783447
GenMatch Time:  8.178052186965942
GenMatch Time:  10.512039422988892
GenMatch Time:  25.145384311676025
GenMatch Time:  22.70319628715515
GenMatch Time:  9.83890700340271
GenMatch Time:  9.47396993637085
GenMatch Time:  10.085129499435425
GenMatch Time:  16.619242668151855
GenMatch Time:  15.874963283538818
GenMatch Time:  9.616381645202637
GenMatch Time:  8.741542100906372
GenMatch Time:  11.944565296173096
GenMatch Time:  14.334772109985352
GenMatch Time: 

In [28]:
run_test_battery(
    est=gm_est,
    runs=gm_runs,
    n_samples=gm_n_samples,
    models=gm_models_sets[2],
    verbosity=2)

Displaying cached results.

Results
Model:  F_mod_nadd_lin
1 Bias:  17.275217121640445
1 RMSE:  0.06910086848656177 

Model:  G_mod_nadd_mod_nlin
1 Bias:  0.26776452898076564
1 RMSE:  0.0010710581159230625 



#### Combine results

In [29]:
results = {}
for file in gm_files_to_be_produced:
    results.update(retrieve_results_dict(file))

results

{'A_add_lin': {'Bias': 20.023235988165702, 'RMSE': 0.08009294395266281},
 'B_add_mild_nlin': {'Bias': 13.717727327601459, 'RMSE': 0.054870909310405835},
 'C_add_mod_nlin': {'Bias': 20.087548601352452, 'RMSE': 0.0803501944054098},
 'D_mild_nadd_lin': {'Bias': 13.127238676118091, 'RMSE': 0.052508954704472366},
 'E_mild_nadd_mild_nlin': {'Bias': 4.4125520199797545,
  'RMSE': 0.017650208079919016},
 'F_mod_nadd_lin': {'Bias': 17.275217121640445, 'RMSE': 0.06910086848656177},
 'G_mod_nadd_mod_nlin': {'Bias': 0.26776452898076564,
  'RMSE': 0.0010710581159230625}}